In [200]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [380]:
init = pd.read_csv('time_wav5.csv', header=None).T
u7 = pd.read_csv('unit7.csv')
u8 = pd.read_csv('unit8.csv')
u13 = pd.read_csv('unit13.csv')
u20 = pd.read_csv('unit8.csv')

startTime = init.iloc[1][0]
endTime = init.iloc[1][1]
print(startTime, endTime)

236880 238080


In [381]:
u7['GPS_time']=u7['GPS_time']-startTime
u8['GPS_time']=u8['GPS_time']-startTime
u13['GPS_time']=u13['GPS_time']-startTime
u20['GPS_time']=u20['GPS_time']-startTime


In [702]:
sampFreq = 65501


u7snaps = pd.read_csv('Unit7PeakTimes.csv', header=None, names = ["dirtime", "reftime"])


u8snaps = pd.read_csv('Unit8PeakTimes.csv', header=None, names = ["dirtime", "reftime"])


u13snaps = pd.read_csv('Unit13PeakTimes.csv', header=None, names = ["dirtime", "reftime"])


u20snaps = pd.read_csv('Unit20PeakTimes.csv', header=None, names = ["dirtime", "reftime"])


In [324]:
from scipy.interpolate import interp1d
import matplotlib.pyplot as plt
time = u7['GPS_time']
xpos = u7['x']
ypos = u7['y']

In [390]:
tnew = np.linspace(0, 1200, num=1201)
xinter = interp1d(time, xpos, kind='nearest')
yinter = interp1d(time, ypos, kind='nearest')

In [426]:
#u7pos is an interpolation from 0 to 1000 
#u7test

In [599]:
u7 = u7[u7['GPS_time'] <= 1100]  
u8 = u8[u8['GPS_time'] <= 1100]  
u13 = u13[u13['GPS_time'] <= 1100]  
u20 = u20[u20['GPS_time'] <= 1100]  


In [600]:
tnew = np.linspace(0, 1000, num=1001)
u7pos = pd.DataFrame({'time':tnew, 'xpos':interp1d(u7['GPS_time'], u7['x'], kind='nearest')(tnew),
                      'ypos':interp1d(u7['GPS_time'], u7['y'], kind='nearest')(tnew)})

u8pos = pd.DataFrame({'time':tnew, 'xpos':interp1d(u8['GPS_time'], u8['x'], kind='nearest')(tnew),
                      'ypos':interp1d(u8['GPS_time'], u8['y'], kind='nearest')(tnew)})

u13pos = pd.DataFrame({'time':tnew, 'xpos':interp1d(u13['GPS_time'], u13['x'], kind='nearest')(tnew),
                      'ypos':interp1d(u13['GPS_time'], u13['y'], kind='nearest')(tnew)})

u20pos = pd.DataFrame({'time':tnew, 'xpos':interp1d(u20['GPS_time'], u20['x'], kind='nearest')(tnew),
                      'ypos':interp1d(u20['GPS_time'], u20['y'], kind='nearest')(tnew)})

In [715]:
u7snaps['dirtimer'] = u7snaps['dirtime'].round()
u7snaps = u7snaps.loc[:, ['dirtime', 'dirtimer']];
u7postime = u7pos.iloc[u7snaps[u7snaps['dirtimer'] < 1000]['dirtimer']]

u8snaps['dirtimer'] = u8snaps['dirtime'].round()
u8snaps = u8snaps.loc[:, ['dirtime', 'dirtimer']];
u8postime = u8pos.iloc[u8snaps[u8snaps['dirtimer'] < 1000]['dirtimer']]

u13snaps['dirtimer'] = u13snaps['dirtime'].round()
u13snaps = u13snaps.loc[:, ['dirtime', 'dirtimer']];
u13postime = u13pos.iloc[u13snaps[u13snaps['dirtimer'] < 1000]['dirtimer']]

u20snaps['dirtimer'] = u20snaps['dirtime'].round()
u20snaps = u20snaps.loc[:, ['dirtime', 'dirtimer']];
u20postime = u20pos.iloc[u20snaps[u20snaps['dirtimer'] < 1000]['dirtimer']]

In [ ]:
direct = pd.DataFrame()

direct['u7dir'] = u7snaps[u7snaps['dirtimer'] < 1000]['dirtime']
direct['u8dir'] = u8snaps[u8snaps['dirtimer'] < 1000]['dirtime']
direct['u13dir'] = u13snaps[u13snaps['dirtimer'] < 1000]['dirtime']
direct['u20dir'] = u20snaps[u20snaps['dirtimer'] < 1000]['dirtime']

direct
dirdiff = pd.DataFrame()
dirdiff['7-8'] = direct['u7dir']-direct['u8dir']
dirdiff['7-13'] = direct['u7dir']-direct['u13dir']
dirdiff['7-20'] = direct['u7dir']-direct['u20dir']
dirdiff['8-13'] = direct['u8dir']-direct['u13dir']
dirdiff['8-20'] = direct['u8dir']-direct['u20dir']
dirdiff['13-20'] = direct['u13dir']-direct['u20dir']

dirdiff.insert(0, "time", u7snaps['dirtimer'])


In [803]:
def getPosAtTime(s):
    u7p = [u7postime.loc[u7postime['time']==s]['xpos'],u7postime.loc[u7postime['time']==s]['ypos']]
    u8p = [u8postime.loc[u8postime['time']==s]['xpos'],u8postime.loc[u8postime['time']==s]['ypos']]
    u13p = [u13postime.loc[u13postime['time']==s]['xpos'],u13postime.loc[u13postime['time']==s]['ypos']]
    u20p = [u20postime.loc[u20postime['time']==s]['xpos'],u20postime.loc[u20postime['time']==s]['ypos']]
    return [u7p, u8p, u13p, u20p]

#getTimeDiff at signal specified at second s
def getTimeDiffAtTime(s):
    return dirdiff.loc[dirdiff['time']==s].values[0][1:7]



In [804]:
auvPos = getPosAtTime(6)
timediff = getTimeDiffAtTime(6)



In [808]:
def getDiffFromShrimp(shrimpPos, auvPos):
    diff = [[0,0],[0,0],[0,0],[0,0]]
    for i in range(len(auvPos)):
        diff[i] = [float(shrimpPos[0] - auvPos[i][0]),float(shrimpPos[1] - auvPos[i][1])];
        
    return diff


def loss(s):
    error = np.zeros(6)
    k = 0
    diff = getDiffFromShrimp(s, auvPos)
    for i in range(4):
        for j in range(i+1,4):
            #print(i,j)
            error[k] = timediff[k]-(np.linalg.norm(diff[i])-np.linalg.norm(diff[j]))
            k = k+1
    return error

from scipy.optimize import least_squares
#shrimp location estimate
initial_estimates=[30,30]
res = least_squares(loss, initial_estimates)
print(res.x[1]/res.x[0])

[-136994.8462432  -347675.95151661]
2.53787613951


In [796]:
float(diff[1][0])

122.595688